# Notebook for preparing the simulations for the coherence map

We prepare two CUPRAD runs that will be used to construct the coherence maps to visualise the phase-matching conditions in the interaction volume. This example will use [pre-ionisation of the medium](https://www.nature.com/articles/s41598-022-11313-6) to reach the phase-matching conditions. 

## Load libraries

In [ ]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import os
import shutil
import h5py
import sys
import copy
import MMA_administration as MMA
import mynumerics as mn
import units
import HHG
from IPython.display import display, Markdown


%matplotlib inline
# import mpld3
# mpld3.enable_notebook()

The directory where the input parameters are written:

In [ ]:
h5path = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'coherence_map','inputs')

## Physical parameters
We set the parameters for the simulation using the reference Gaussian beam (see [notebook 1](../gas_cell/prepare_cell.ipynb) and [notebook 2](../density_profile/prepare_density_profile.ipynb) for more details). We already specify the optimised harmonic order, wchich will be used to choose the degree of pre-ionisation. 

In [ ]:
Horder = 17

### Medium parameters

Additionally to the IR propgation, we need to provide the XUV tables to address the phase matching condition for the selected harmonic.

In [ ]:
# gas specifiers
gas = 'Kr'
medium_length = 15e-3 # [m]
ionisation_model = 'PPT'
XUV_dispersion_tables = 'NIST'
medium_pressure = 25e-3 # bar

# pre-ionisation degree relative to the optimal ionisation for the phase-matching (see https://www.nature.com/articles/s41598-022-11313-6)
# (there will be some ionisation due to the laser)
pre_ionisation_degree = 0.9 

### Laser parameters

The parmaters of [the reference Gaussian beam](../gas_cell/prepare_cell.ipynb) focused in the middle of the cell.

In [ ]:
reference_Gaussian_focus = medium_length/2. # [m]
reference_Gaussian_waist = 100e-6 # [m]
reference_Gaussian_focus_intensity = 1.3e18 # [W/m2]
laser_wavelength = 800e-9 # [m]
laser_pulse_duration = 45e-15 # [s] (defined via 1/e in the electric field amplitude)

### Compute the optimal pre-ionisation degree
The `HHG` module provides the routine for the optimal ionisation degree. It evaluates the phase matching condition $\Delta k = qk_1 - k_q \overset{!}{=}0$.



In [ ]:
## compute the optimal ionisation degree
import XUV_refractive_index as XUV_index
import IR_refractive_index as IR_index
omegaSI = mn.ConvertPhoton(laser_wavelength,'lambdaSI','omegaSI')
eta_opt = HHG.eta_opt(omegaSI,
                    XUV_index.polarisability(Horder * omegaSI, gas+'_'+XUV_dispersion_tables,),
                    IR_index.polarisability(gas,laser_wavelength))

### Numerical parameters

In [ ]:
number_of_points_in_r      = 1024
number_of_points_in_t      = 1024

operators_t                = 2
first_delta_z              = 0.01 # [mm]
phase_threshold_for_decreasing_delta_z = 0.002	# [rad]

length_of_window_for_r_normalized_to_beamwaist = 4.   # [-]
length_of_window_for_t_normalized_to_pulse_duration = 6. # [-]

number_of_absorber_points_in_time = 16  # [-]

physical_output_distance_for_plasma_and_Efield = 0.00001   # [m]

output_distance_in_z_steps_for_fluence_and_power   = 100  # [-]

radius_for_diagnostics = 0.1 # [cm]

run_time_in_hours = 5.0 # [h] 

In [ ]:
## Code to generate the following text ##
zR = (np.pi*reference_Gaussian_waist**2)/laser_wavelength
dr_CUPRAD = length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)/number_of_points_in_r
display(Markdown(rf"""### Properties of the chosen discretisation
* The chosen discretisation in time gives ~ {
            number_of_points_in_t/(
            laser_pulse_duration*length_of_window_for_t_normalized_to_pulse_duration/mn.ConvertPhoton(laser_wavelength,'lambdaSI','T0SI')
            )
    :.0f}
points per one laser period.
* The stepsize in the radial discretisation is ~ ${
      1e6*dr_CUPRAD
      :.2f}
~\mu {{\mathrm{{m}}}}$.
* The size of the radial computational box is ~ ${
      1e6*length_of_window_for_r_normalized_to_beamwaist * reference_Gaussian_waist
      :.2f}
~\mu {{\mathrm{{m}}}}$. The maximal radius of the reference Gaussian beam is ~ ${
      1e6*np.max([
            reference_Gaussian_waist*np.sqrt(1+((medium_length-reference_Gaussian_focus)/zR)**2),
            reference_Gaussian_waist*np.sqrt(1+(reference_Gaussian_focus/zR)**2)
            ])
      :.2f}
~\mu {{\mathrm{{m}}}}$.$^\dagger$
* The Rayleigh length for the purely Gaussian beam is ${
      1e3*zR
      :.2f}
~{{\mathrm{{mm}}}}$ (the length of the cell is ${
      1e3*medium_length
      :.2f}
~{{\mathrm{{mm}}}}$).
* There will be ~ {
    medium_length/physical_output_distance_for_plasma_and_Efield
    :.0f} output planes.

$^\dagger$ This is given at the $z$-edges of the computational box.
"""))





## Prepare the input files

Finally, the inputs files are created. Additionally to [the standard operation](../gas_cell/prepare_cell.ipynb), we add the subgroup driving the pre-ionisation within the `global_inputs`.

In [ ]:
# Code to create the input hdf5-file
## First, we prepare dictionaries between hdf5-inputs and this jupyter notebook


pre_ion_path =  MMA.global_inputs_pre_ionised_subgroup
global_input_names_to_jupyter_variables = {
    'gas_preset'                                : (np.bytes_(gas),                        '[-]'   ),
    'medium_pressure_in_bar'                    : (medium_pressure,                       '[bar]' ),

    pre_ion_path + '/method_geometry'           : (1,                                        '[-]'),
    pre_ion_path + '/method_units'              : (1,                                        '[-]'),
    pre_ion_path + '/initial_electrons_ratio'   : (0.,                                       '[-]'),
   
}


CUPRAD_names_to_jupyter_variables = {
    # laser parameters
    'laser_wavelength'                          : (1e2*laser_wavelength,                  '[cm]'  ),
    'laser_pulse_duration_in_1_e_Efield'        : (1e15*laser_pulse_duration,             '[fs]' ),
    'laser_focus_intensity_Gaussian'            : (reference_Gaussian_focus_intensity,    '[W/m2]'  ),
    'laser_focus_beamwaist_Gaussian'            : (reference_Gaussian_waist,              '[m]'  ),
    'laser_focus_position_Gaussian'             : (reference_Gaussian_focus,              '[m]'  ),

    # medium parameters
    'medium_physical_distance_of_propagation'   : (medium_length,                         '[m]'   ),

    # ionisation
    'ionization_model'                          : (np.bytes_(ionisation_model),          '[-]'  ),

    # numerics
    'numerics_number_of_points_in_r'            : (number_of_points_in_r,                 '[-]'  ),
    'numerics_number_of_points_in_t'            : (number_of_points_in_t,                 '[-]'  ),
    'numerics_operators_t_t-1'                  : (operators_t,                           '[-]'  ),
    'numerics_physical_first_stepwidth'         : (first_delta_z,                         '[mm]' ),
    'numerics_phase_threshold_for_decreasing_delta_z' : 
        (phase_threshold_for_decreasing_delta_z,                '[rad]' ),
    'numerics_length_of_window_for_r_normalized_to_beamwaist':
        (length_of_window_for_r_normalized_to_beamwaist,        '[-]'   ),
    'numerics_length_of_window_for_t_normalized_to_pulse_duration' :
        (length_of_window_for_t_normalized_to_pulse_duration,   '[-]'   ),
    'numerics_number_of_absorber_points_in_time':
        (number_of_absorber_points_in_time ,                    '[-]'   ),
    'numerics_physical_output_distance_for_plasma_and_Efield' :
        (physical_output_distance_for_plasma_and_Efield,        '[m]'   ),
    'numerics_output_distance_in_z-steps_for_fluence_and_power' :
        (output_distance_in_z_steps_for_fluence_and_power,      '[-]'   ),
    'numerics_radius_for_diagnostics'           : (radius_for_diagnostics,                '[cm]' ),
    'numerics_run_time_in_hours'                : (run_time_in_hours,                     '[s]'  )
}


# The inputs for the pre-ionised case are the same except the pre-ionisation 
global_input_names_to_jupyter_variables2 = copy.deepcopy(global_input_names_to_jupyter_variables)
# set the pre-ionisation
global_input_names_to_jupyter_variables2[pre_ion_path + '/initial_electrons_ratio'] = (pre_ionisation_degree*eta_opt, '[-]') 


## Create the hdf5-archive

# clean the input directory if it existed
if os.path.exists(h5path): shutil.rmtree(h5path) 
os.makedirs(h5path)

h5filename = 'results_map1.h5'
h5filepath = os.path.join(h5path,h5filename)

h5filename2 = 'results_map2.h5'
h5filepath2 = os.path.join(h5path,h5filename2)

# create the files
from inputs_transformer import add_variables2hdf5, variables2text
with h5py.File(h5filepath,'w') as f1, h5py.File(h5filepath2,'w') as f2: 

    add_variables2hdf5(f1,
                    global_input_names_to_jupyter_variables,
                    CUPRAD_names_to_jupyter_variables,
                    None,
                    None,
                    None,
                    None)

    add_variables2hdf5(f2,
                    global_input_names_to_jupyter_variables2,
                    CUPRAD_names_to_jupyter_variables,
                    None,
                    None,
                    None,
                    None)